In [ ]:
# System
import gc
import os
import sys
from pathlib import Path

# Cpu
from collections import Counter
import numpy as np
import pandas as pd

# Data viz
from IPython.display import Image, display, HTML
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pprint
import seaborn as sns
plt.rcParams['legend.framealpha'] = 1
plt.rcParams['legend.frameon'] = True
plt.rcParams['legend.edgecolor'] = 'k'
plt.rcParams['legend.fancybox'] = False
plt.rcParams['legend.facecolor'] = 'w'
plt.rcParams['legend.handlelength'] = 1
plt.rcParams['legend.handleheight'] = 1.125
plt.rcParams['figure.figsize'] = (16, 9)


display(HTML("<style>.container { width:95% !important; }</style>"))

# Plot modes in (a, b) space

In [ ]:
PROJECT_DIR = ""

In [ ]:
root = Path(PROJECT_DIR)
p = root / 'analysis'

files = sorted(p.glob('**/*.tsv'))
files = list(filter(lambda f: 'global' not in f.name, files))
manuscripts = list(map(lambda f: f.parent.name, files))
illuminations = sorted((root / 'raw_data').glob('**/*.jpg'))

In [ ]:
from ast import literal_eval
df = pd.DataFrame()
dtype = {
    'image_name': str,
    'bandwidth': float,
    'nb_cluster': int,
    'cluster_centers': str,
}
for f in files:
    tmp_df = pd.read_csv(f, sep='\t', header=0, dtype=dtype)
    tmp_df['manuscript'] = f.parent.name
    df = pd.concat([df, tmp_df], ignore_index=True)
    
df['centers'] = df['cluster_centers'].apply(literal_eval)
df['centers'] = df['centers'].apply(lambda l: l[:4])
df

In [ ]:
from matplotlib.colors import ListedColormap
import cv2

with sns.axes_style('white', {'axes.grid': True}):
    fig, ax = plt.subplots(figsize=(10, 10))
    centers = np.asarray(df['centers'].sum())
    centers_lab = np.hstack([[[100]]*len(centers), centers])
    colors_list = cv2.cvtColor(np.reshape(centers_lab, (1, -1, 3)).astype('uint8'), cv2.COLOR_Lab2RGB)[0] / 255.
    ax.scatter(*list(zip(*centers)), c=colors_list, s=200, marker='o', edgecolor=[(0, 0, 0, 1)]*len(colors_list))
    ax.set_xlabel('a')
    ax.set_ylabel('b')
    ax.set_xlim((90, 190))
    ax.set_ylim((90, 190))
    ax.set_title('Cluster centers in (a, b) color space', fontweight='bold', size=14)
    fig.tight_layout()
#     fig.savefig('scatter_cluster_centers.jpg')

# Zoom on manuscripts

## yellow cluster

In [ ]:
zoom_centers = centers[(centers[:, 1] > 159) & (centers[:, 1] < 165) & (centers[:, 0] > 132) & (centers[:, 0] < 138)]
zoom_df = df[df['centers'].apply(lambda l: max([e in zoom_centers for e in l]))]

In [ ]:
margin = 0.5
with sns.axes_style('white', {'axes.grid': True}):
    fig, ax = plt.subplots(figsize=(10, 10))
    for name, possible_centers in zip(zoom_df['image_name'], zoom_df['centers']):
        a, b = [e for e in possible_centers if e in zoom_centers][0]
        f = list(filter(lambda f: f.name == name, illuminations))[0]
        ax.imshow(Image.open(f), extent=(a - margin, a + margin, b - margin, b + margin))
    ax.set_xlabel('a')
    ax.set_ylabel('b')
    x_min, y_min = (zoom_centers.min(axis=0) - 3*margin).astype(np.uint8)
    x_max, y_max = (zoom_centers.max(axis=0) + 3*margin).astype(np.uint8)
    ax.set_xlim((x_min, x_max))
    ax.set_ylim((y_min, y_max))
    fig.tight_layout()
#     fig.savefig('yellow_cluster_ex.jpg')

## blue cluster

In [ ]:
zoom_centers = centers[(centers[:, 1] < 110) & (centers[:, 0] > 125)]
zoom_df = df[df['centers'].apply(lambda l: max([e in zoom_centers for e in l]))]

In [ ]:
margin = 0.4
with sns.axes_style('white', {'axes.grid': True}):
    fig, ax = plt.subplots(figsize=(10, 10))
    for name, possible_centers in zip(zoom_df['image_name'], zoom_df['centers']):
        a, b = [e for e in possible_centers if e in zoom_centers][0]
        f = list(filter(lambda f: f.name == name, illuminations))[0]
        ax.imshow(Image.open(f), extent=(a - margin, a + margin, b - margin, b + margin))
    ax.set_xlabel('a')
    ax.set_ylabel('b')
    x_min, y_min = (zoom_centers.min(axis=0) - 3*0.5 + 1).astype(np.uint8)
    x_max, y_max = (zoom_centers.max(axis=0) + 3*0.5).astype(np.uint8)
    ax.set_xlim((x_min, x_max))
    ax.set_ylim((y_min, y_max))
    fig.tight_layout()
#     fig.savefig('blue_cluster_ex.jpg')